In [3]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages (0.0)


In [4]:
# install joblib for saving
# Restart kernel after installing 
!pip install joblib

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
import missingno as msno
import seaborn as sns

In [6]:
df = pd.read_csv("../data/ml_compile.csv")

## Categorical Algorithms
### Preprocessing - Transform RSF values to Categorical

In [7]:
df["bear_rsf_mean"].fillna(0, inplace=True)

In [8]:
conditions = [(df["bear_rsf_mean"] < 6), (df["bear_rsf_mean"] > 6) & (df["bear_rsf_mean"] < 10), (df["bear_rsf_mean"] > 10) & (df["bear_rsf_mean"] < 15), (df["bear_rsf_mean"]>15)]
values = ['unknown','low', 'mid', 'high']
df["bear_mean_cat"] = np.select(conditions,values)

In [11]:
#Change Threshold to 9
df_cat_null = df [['land_avg_temp', 'land_max_temp', 'land_min_temp',
       'land_ocean_avg_temp', 'north_min_temp_anomoly',
       'north_max_temp_anomoly', 'north_mean_temp_anomoly', 'global_avg_co2',
       'seaice_extent','bear_mean_cat']]

In [12]:
df_cat_null = df_cat_null.dropna(axis=0, how="any")

In [13]:
df_cat_null.describe()

,land_avg_temp,land_max_temp,land_min_temp,land_ocean_avg_temp,north_min_temp_anomoly,north_max_temp_anomoly,north_mean_temp_anomoly,global_avg_co2,seaice_extent
count,447.000000,447.000000,447.000000,447.000000,447.000000,447.000000,447.000000,447.000000,447.000000
mean,9.226651,14.930488,3.615477,15.641613,0.775022,0.659465,0.686273,366.108054,11.559153
std,4.134906,4.249913,4.010169,1.229699,0.528649,0.567605,0.540999,18.908570,3.185538
min,2.558000,8.071000,-2.853000,13.566000,-0.787000,-1.256000,-1.065000,332.410000,3.565600
25%,5.189000,10.696000,-0.308000,14.441000,0.413500,0.253000,0.289000,350.320000,8.842585
50%,9.394000,15.174000,3.759000,15.683000,0.766000,0.653000,0.671000,364.310000,12.199871
75%,13.185500,19.062000,7.447000,16.801500,1.101500,1.037000,1.029500,382.250000,14.409063
max,15.482000,21.320000,9.715000,17.611000,2.554000,2.465000,2.510000,403.960000,16.341938


### Assign new variables

In [ ]:
X = df_cat_null[['land_avg_temp', 'land_max_temp', 'land_min_temp',
       'land_ocean_avg_temp', 'north_min_temp_anomoly',
       'north_max_temp_anomoly', 'north_mean_temp_anomoly', 'global_avg_co2',
       'seaice_extent']]
y_cat = df_cat_null["bear_mean_cat"]
print(X.shape, y_cat.shape)

### Split the data into testing and training dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_cat_train, y_cat_test = train_test_split(X, y_cat, test_size=0.5, random_state=42)

### MinMaxScalar to fit and transform X features and y target

In [ ]:
#Fit Transform using MinMaxScalar for X features
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)

## 2. Random Forest Analysis

In [ ]:
from sklearn.ensemble import RandomForestClassifier
y_cat_rf = df_cat_null["bear_mean_cat"]
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_minmax, y_cat_train)
score = rf.score(X_test_minmax, y_cat_test)


In [ ]:
rf.get_params().keys()

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y_cat_train)
encoded_y_train = label_encoder.transform(y_cat_train)
encoded_y_test = label_encoder.transform(y_cat_test)

In [ ]:
rf2 = RandomForestClassifier(n_estimators=200, min_samples_split =5, min_samples_leaf = 4, max_features='auto', max_depth=60, bootstrap='True')
rf2 = rf2.fit(X_train_minmax, encoded_y_train)
score = rf2.score(X_test_minmax, encoded_y_test)